In [ ]:
import pandas as pd
import numpy as np

### Carga inicial de los datos

In [ ]:
# Create Pandas DataFrame from CSV file
df_music = pd.read_csv('./Iconic_Songs_Dataset.csv')
df_music.head(3)

,Title,Artist,Genre,ReleaseDate,Duration,Album,Popularity
0,Blinding Lights,The Weeknd,Pop,2024-01-05,3:22,After Hours,95
1,Levitating,Dua Lipa,Pop,2024-01-14,3:23,Future Nostalgia,93
2,Peaches,Justin Bieber ft. Daniel Caesar & Giveon,Pop,2024-02-12,3:18,Justice,91


For now, for our test database, we are interested in the following attributes:

- **'Title'**: The name of the song.
- **'Artist'**: The name of the artist or band who performed the song.
- **'Genre'**: The genre of the song, such as Pop, K-Pop, Afrobeats, etc.
- **'ReleaseDate'**: The date the song was released.
- **'Duration'**: The length of the song in minutes and seconds.

The **'Popularity'** attribute will also be useful to randomly assign a play count based on each song's popularity. This approach allows us to work with this dataset without needing the exact play counts for each song, as this is simply a test for a recommendation system using a basic SQL database.

### Pre-processing and cleaning

In [ ]:
print(df_music.shape)

(100, 7)


In [ ]:
df_music.dtypes

,0
Title,object
Artist,object
Genre,object
ReleaseDate,object
Duration,object
Album,object
Popularity,int64


#### We reduce the DataFrame to have only the desired attributes

In [ ]:
# Drop columns that will not be used
df_music.drop(['Album'], axis=1, inplace=True)
df_music.head(3)

,Title,Artist,Genre,ReleaseDate,Duration,Popularity
0,Blinding Lights,The Weeknd,Pop,2024-01-05,3:22,95
1,Levitating,Dua Lipa,Pop,2024-01-14,3:23,93
2,Peaches,Justin Bieber ft. Daniel Caesar & Giveon,Pop,2024-02-12,3:18,91


#### We modify the DataFrame to work with the units we are interested in

- **ReleaseDate**: The date presents the desired format, but we will keep only the year.
- **Duration**: We want the duration to be in seconds.
- **Popularity**: Popularity could represent total streams.

In [ ]:
# Function to convert date format YYYYY-MM-DD to YYYYY
from datetime import datetime

def convert_to_year(release_date):
  year = datetime.strptime(release_date, "%Y-%m-%d").year
  return year

# Apply the function to the column 'ReleaseDate', which will be renamed 'Year'
df_music.rename(columns={'ReleaseDate': 'Year'}, inplace=True)
df_music['Year'] = df_music['Year'].apply(convert_to_year)
df_music.head(3)

,Title,Artist,Genre,Year,Duration,Popularity
0,Blinding Lights,The Weeknd,Pop,2024,3:22,95
1,Levitating,Dua Lipa,Pop,2024,3:23,93
2,Peaches,Justin Bieber ft. Daniel Caesar & Giveon,Pop,2024,3:18,91


In [ ]:
# Function to convert from 'minutes:seconds' to seconds
def convert_to_seconds(duration):
  minutes, seconds = map(int, duration.split(':'))
  return minutes*60 + seconds

# Apply the function to the column 'Duration'
df_music['Duration'] = df_music['Duration'].apply(convert_to_seconds)
df_music.head(3)

,Title,Artist,Genre,Year,Duration,Popularity
0,Blinding Lights,The Weeknd,Pop,2024,202,95
1,Levitating,Dua Lipa,Pop,2024,203,93
2,Peaches,Justin Bieber ft. Daniel Caesar & Giveon,Pop,2024,198,91


For popularity, since we are only working with test data that doesn't need to be real, we can consider popularity as a value between 0 and 1 (for example, a popularity score of 95 would be 0.95). Then, we can multiply this value by one billion and use the result (0.95 * 1,000,000,000).

In [ ]:
# Function to convert from popularity rating to millions of streams
def convert_to_millions(popularity):
  return round((popularity/100) * 1000000000)

# Apply the function to the column
df_music['Popularity'] = df_music['Popularity'].apply(convert_to_millions)
df_music.head(3)

,Title,Artist,Genre,Year,Duration,Popularity
0,Blinding Lights,The Weeknd,Pop,2024,202,950000000
1,Levitating,Dua Lipa,Pop,2024,203,930000000
2,Peaches,Justin Bieber ft. Daniel Caesar & Giveon,Pop,2024,198,910000000


Regarding the basic attributes of this DataFrame, we would only need to be able to assign a random and unique **'id'** to each of the songs that appear in the DataFrame.

In [ ]:
# Assign unique ID to each song
df_music['id'] = np.random.permutation(len(df_music)) + 1  # Starts from 1

In [ ]:
# Unique artists there are to identify the existence of collaborations
print(df_music['Artist'].unique())

['The Weeknd' 'Dua Lipa' 'Justin Bieber ft. Daniel Caesar & Giveon'
 'Olivia Rodrigo' 'The Kid LAROI & Justin Bieber' 'Doja Cat ft. SZA'
 'Lil Nas X' 'BTS' 'Lil Nas X & Jack Harlow' 'Billie Eilish' 'Ed Sheeran'
 'Adele' 'Coldplay & BTS' 'Doja Cat' 'Mimi Webb' 'Elton John & Dua Lipa'
 'Glass Animals' 'Justin Bieber' 'CKay' 'Walker Hayes' 'WizKid ft. Tems'
 'Justin Bieber & Benny Blanco' 'Taylor Swift' 'Ariana Grande'
 'Post Malone' 'Harry Styles' 'Chris Brown & Young Thug'
 'Ariana Grande & Justin Bieber' 'Lewis Capaldi' 'SAINt JHN' 'Drake'
 'Saweetie ft. Doja Cat' 'Pop Smoke ft. Lil Baby & DaBaby'
 'Post Malone & Swae Lee' 'Drake ft. Lil Durk' 'Maroon 5'
 'Lady Gaga & Ariana Grande' 'Giveon' 'Lil Mosey'
 'Jawsh 685 & Jason Derulo' 'Lizzo' 'Arizona Zervas'
 'Shawn Mendes & Camila Cabello'
 'Megan Thee Stallion ft. Nicki Minaj & Ty Dolla $ign' 'Jonas Brothers'
 'Chris Brown ft. Drake' 'SHAED' 'Marshmello & Bastille' 'Halsey'
 'Benny Blanco & Halsey & Khalid' 'Juice WRLD' 'Sheck Wes' 'Tra

We can observe the presence of characters like: 'ft.' and '&' used to separate artist names. For example:

- Justin Bieber ft. Daniel Caesar & Giveon

It would be more appropriate, for songs like this, to split the data into three separate rows where the key is the 'id'.

In [ ]:
import re

# Function to split the 'Artist' column correctly
def split_artists(artist_str):
  return re.split(r' ft\. |\s&\s|, ', artist_str)  # Divide by 'ft.', ' & ', y ', '

# We apply the function to the DataFrame and "explode" rows
df_music_exploded = df_music.assign(Artist=df_music['Artist'].apply(split_artists)).explode('Artist').reset_index(drop=True)
  # apply(split_artists) → Applies the function to each row of the 'Artist' column, transforming it into a list of artists.
  # explode('Artist') → Expands the DataFrame, creating a new row for each artist in the collaboration.
  # reset_index(drop=True) → Reset the indexes to avoid inconsistencies.
df_music_exploded.head(5)

,Title,Artist,Genre,Year,Duration,Popularity,id
0,Blinding Lights,The Weeknd,Pop,2024,202,950000000,53
1,Levitating,Dua Lipa,Pop,2024,203,930000000,74
2,Peaches,Justin Bieber,Pop,2024,198,910000000,11
3,Peaches,Daniel Caesar,Pop,2024,198,910000000,11
4,Peaches,Giveon,Pop,2024,198,910000000,11


#### We create a new DataFrame for artists to assign their 'id'

As we can see in the original table, it is necessary to link the **SONG** table with the **ARTIST** table. Therefore, we will assign random **'id'** values to each artist appearing in **'df_music'** through another DataFrame called **'df_artists'**.

In [ ]:
# Copy the artists to another DataFrame
df_artists = df_music_exploded[['Artist']].rename(columns={'Artist': 'name'}).copy()  # Double brackets are used to maintain the DataFrame format and not to get a Series
  # copy() → Ensures that the original DataFrame (df_music_exploded) is not modified.

In [ ]:
# Obtain unique artists
unique_artists = df_music_exploded['Artist'].drop_duplicates().reset_index(drop=True)

# Create DataFrame for unique artists
df_artists = pd.DataFrame({'name': unique_artists})

# Generate random unique IDs
df_artists['artist_id'] = np.random.permutation(len(df_artists)) + 1  # Starts from 1
df_artists.head(3)

,name,artist_id
0,The Weeknd,60
1,Dua Lipa,53
2,Justin Bieber,22


It is interesting to note that the 'id' values could have been assigned earlier in **'df_music_exploded'**, but following a workflow more aligned with SQL database practices, the 'id' data should come from the **'df_artists'** DataFrame (table), as it contains the information related to the artists.

For this reason, we will assign the data to a new column called 'id_artist' in the **'df_music_exploded'** DataFrame.

In [ ]:
# Merge by artist name
df_music_exploded = df_music_exploded.merge(df_artists, left_on='Artist', right_on='name', how='left')

# Remove duplicate column 'name' after merge
df_music_exploded.drop(columns='name', inplace=True)
df_music_exploded.head(5)

,Title,Artist,Genre,Year,Duration,Popularity,id,artist_id
0,Blinding Lights,The Weeknd,Pop,2024,202,950000000,53,60
1,Levitating,Dua Lipa,Pop,2024,203,930000000,74,53
2,Peaches,Justin Bieber,Pop,2024,198,910000000,11,22
3,Peaches,Daniel Caesar,Pop,2024,198,910000000,11,11
4,Peaches,Giveon,Pop,2024,198,910000000,11,70


Now, we need two more attributes in our **'df_artists'** DataFrame:

- **country**: If we want real data, we can use an AI tool to obtain the information or fill it in manually.
- **followers**: This can be randomly estimated within a range that is somewhat proportional to the popularity of their songs. However, this doesn't need to be strongly correlated, so we will opt for a simpler approach.

With this in mind, we will leave the **'country'** attribute for later. Moving on with the **'followers'** column, we will apply the following function to the DataFrame.

In [ ]:
# Function to randomly assign followers to artists
def assign_followers_simple(df):
  df['followers'] = np.random.randint(10_000_000, 100_000_000, size=len(df))
  return df

df_artists = assign_followers_simple(df_artists)
df_artists.head(3)

,name,artist_id,followers
0,The Weeknd,60,67886848
1,Dua Lipa,53,31157517
2,Justin Bieber,22,36398525


In [ ]:
# Download the DataFrame as a .csv file to add the column 'country'
#df_artists = df_artists.drop(columns=['artist_id', 'followers'])
#df_artists.to_csv('artists_no_country.csv', index=False)

In [ ]:
# We load the modified DataFrame with the new column
df_artists_modified = pd.read_csv('./artists_country.csv')
df_artists_modified.head(3)

,name,country
0,The Weeknd,CAN
1,Dua Lipa,UK
2,Justin Bieber,CAN


In [ ]:
print(df_artists_modified['country'].unique())

['CAN' 'UK' 'USA' 'AUS' 'KOR' 'NGA' 'NZ' 'CUB' 'TT' 'PR' 'COL' 'FRA']


It is necessary to re-add the 'id' and 'followers' columns that were in the previous DataFrame, so that the consistency of the randomly generated numbers in each run is maintained. This allows to avoid that the values as 'id' are different when downloading and uploading the artists file in each execution.

In [ ]:
# Re-add 'id' and 'followers'
df_artists_modified[['id', 'followers']] = df_artists[['artist_id', 'followers']]

# We check the results
df_artists_modified.head(5)

,name,country,id,followers
0,The Weeknd,CAN,60,67886848
1,Dua Lipa,UK,53,31157517
2,Justin Bieber,CAN,22,36398525
3,Daniel Caesar,CAN,11,49435734
4,Giveon,USA,70,64492856


We already have the DataFrames that would represent the tables **ARTIST** (df_artists_modified) and **SONG** (df_music_exploded) of our database. Now, the only thing left to do is to change the names and order of the columns to have everything more orderly.

In [ ]:
# Rearrange the columns
df_music_exploded = df_music_exploded.reindex(columns=['id','Title','Genre','Duration','Popularity','Year','artist_id'])
df_artists_modified = df_artists_modified.reindex(columns=['id','name','country','followers'])

# Change the column names
df_music_exploded = df_music_exploded.rename(columns={'Title': 'title', 'Genre': 'genre', 'Duration': 'duration', 'Popularity': 'streams', 'Year': 'year'})

In [ ]:
# We check the results
df_music_exploded.head(5)

,id,title,genre,duration,streams,year,artist_id
0,53,Blinding Lights,Pop,202,950000000,2024,60
1,74,Levitating,Pop,203,930000000,2024,53
2,11,Peaches,Pop,198,910000000,2024,22
3,11,Peaches,Pop,198,910000000,2024,11
4,11,Peaches,Pop,198,910000000,2024,70


Because the SONG table cannot have more than one 'id' repeated. It is necessary to create an intermediate table that we will call **SING**, which has both 'song_id' and 'artist_id' as a composite primary key. Therefore, it is necessary to create a new DataFrame.

In [ ]:
# Create the intermediate DataFrame with the required columns
df_sing = df_music_exploded[['id', 'artist_id']].rename(columns={'id': 'song_id'}).drop_duplicates()

# Display the first values of the intermediate table
df_sing.head(3)

,song_id,artist_id
0,53,60
1,74,53
2,11,22


In [ ]:
# Delete column 'artist_id' and remove duplicates based on 'song_id' in 'df_music_exploded'
df_music_exploded = df_music_exploded.drop(columns=['artist_id']).drop_duplicates(subset=['id'])

# We check the results
df_music_exploded.head(3)

,id,title,genre,duration,streams,year
0,53,Blinding Lights,Pop,202,950000000,2024
1,74,Levitating,Pop,203,930000000,2024
2,11,Peaches,Pop,198,910000000,2024


### Exportar los DataFrames en formato CSV

In [ ]:
# Download DataFrames as .csv files
df_music_exploded.to_csv('song.csv', index=False)
df_artists_modified.to_csv('artist.csv', index=False)
df_sing.to_csv('sing.csv', index=False)